# PRIMERA PARTE (Conexión a Redshift)

### Alfonso Pinto CoderHouse

In [1]:
# Importo librerias
# pip install psycopg2 pandas python-dotenv
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv
from psycopg2.extras import execute_values


In [2]:
load_dotenv()

# Se cargan las credenciales, a partir de un archivo .env, de la base de datos y se realiza la conexión.

CODER_REDSHIFT_HOST = os.environ.get('CODER_REDSHIFT_HOST')
CODER_REDSHIFT_DB = os.environ.get('CODER_REDSHIFT_DB')
CODER_REDSHIFT_USER = os.environ.get('CODER_REDSHIFT_USER')
CODER_REDSHIFT_PASS = os.environ.get('CODER_REDSHIFT_PASS')
CODER_REDSHIFT_PORT = os.environ.get('CODER_REDSHIFT_PORT')

try:
    conn = psycopg2.connect(
        host=CODER_REDSHIFT_HOST,
        dbname=CODER_REDSHIFT_DB,
        user=CODER_REDSHIFT_USER,
        password=CODER_REDSHIFT_PASS,
        port=CODER_REDSHIFT_PORT,

    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


# SEGUNDA PARTE (Conexión y Parsing de Datos a partir de API AlphaVantage)

In [18]:
import requests

alphavantage_api_key = os.environ.get('ALPHAVANTAGE_API_KEY')

In [24]:
def get_json(symbol):
    # Se realiza la petición a la API de Alpha Vantage con el símbolo de la acción y la llave de la API
    # La llave de la API se encuentra en el archivo .env y es gratuita, se puede obtener en https://www.alphavantage.co/support/#api-key.
    # AL no tener opciones de pago, la API solo permite 5 peticiones por minuto y 500 peticiones por día.
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&interval=5min&apikey={alphavantage_api_key}"
    r = requests.get(url)
    print(r)
    return r.json()
def json_a_diccionario(json):
    # Se crea un diccionario vacío
    diccionario = {}
    # Se itera sobre las llaves del json
    for llave in json.keys():
        diccionario[llave] = json[llave]
    return diccionario
def format_json(json, symbol):
    # Se crea un DataFrame en Pandas a partir del json y se transpone para que las columnas sean los datos y las filas los días
    df = pd.DataFrame(json['Monthly Adjusted Time Series']).T
    # Se cambian los nombres de las columnas para que no tengan enumeración
    df.rename(columns=lambda x: x[3:], inplace=True)
    # Agregar columna index
    df['date'] = df.index
    df.reset_index(drop=True, inplace=True)
    # Se definen los tipos de datos de las columnas ya que naturalmente son todos strings VARCHAR
    df['date'] = pd.to_datetime(df['date'])
    df['open'] = pd.to_numeric(df['open'])
    df['high'] = pd.to_numeric(df['high'])
    df['low'] = pd.to_numeric(df['low'])
    df['close'] = pd.to_numeric(df['close'])
    df['adjusted close'] = pd.to_numeric(df['adjusted close'])
    # Cambiar volumen a millones
    df['volume'] = pd.to_numeric(df['volume'])
    df['volume'] = round(df['volume'] / 1000000)
    df['volume'] = df['volume'].astype(int)
    df['dividend amount'] = pd.to_numeric(df['dividend amount'])
    #Se agrega la columna symbol con el símbolo de la acción
    df['symbol'] = symbol
    df['symbol'] = df['symbol'].astype(str)
    #Se devuelve el DataFrame ya transformado
    return df

In [23]:
# Apple
data = get_json('AAPL')
dict = json_a_diccionario(data)
dict = dict['Monthly Adjusted Time Series'] #No nos interesa la metadata por lo que solo seleccionamos el diccionario que contiene los datos
dict.keys()
#Nos encontramos con que el diccionario tiene como llaves las fechas y como valores los datos de cada fecha por lo que procedemos a convertirlo en un dataframe para luego transponerlo y poder trabajar con el.

<Response [200]>


dict_keys(['2023-10-11', '2023-09-29', '2023-08-31', '2023-07-31', '2023-06-30', '2023-05-31', '2023-04-28', '2023-03-31', '2023-02-28', '2023-01-31', '2022-12-30', '2022-11-30', '2022-10-31', '2022-09-30', '2022-08-31', '2022-07-29', '2022-06-30', '2022-05-31', '2022-04-29', '2022-03-31', '2022-02-28', '2022-01-31', '2021-12-31', '2021-11-30', '2021-10-29', '2021-09-30', '2021-08-31', '2021-07-30', '2021-06-30', '2021-05-28', '2021-04-30', '2021-03-31', '2021-02-26', '2021-01-29', '2020-12-31', '2020-11-30', '2020-10-30', '2020-09-30', '2020-08-31', '2020-07-31', '2020-06-30', '2020-05-29', '2020-04-30', '2020-03-31', '2020-02-28', '2020-01-31', '2019-12-31', '2019-11-29', '2019-10-31', '2019-09-30', '2019-08-30', '2019-07-31', '2019-06-28', '2019-05-31', '2019-04-30', '2019-03-29', '2019-02-28', '2019-01-31', '2018-12-31', '2018-11-30', '2018-10-31', '2018-09-28', '2018-08-31', '2018-07-31', '2018-06-29', '2018-05-31', '2018-04-30', '2018-03-29', '2018-02-28', '2018-01-31', '2017-12-

In [5]:
df_aapl = format_json(data, 'AAPL')
df_aapl.head(2)

<Response [200]>


,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,171.220,179.85,170.82,179.80,179.80,394,0.0,2023-10-11,AAPL
1,189.485,189.98,167.62,171.21,171.21,1338,0.0,2023-09-29,AAPL


In [7]:
# Amazon
data = get_json('AMZN')
df_amzn = format_json(data, 'AMZN')
df_amzn.head(2)

<Response [200]>


,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,127.280,132.05,124.13,131.83,131.83,352,0.0,2023-10-11,AMZN
1,139.455,145.86,123.04,127.12,127.12,1121,0.0,2023-09-29,AMZN


In [25]:
# Google
data = get_json('GOOG')
df_goog = format_json(data, 'GOOG')
df_goog.head(2)

<Response [200]>


,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,132.155,142.22,132.065,141.70,141.70,155,0.0,2023-10-11,GOOG
1,138.430,139.93,128.190,131.85,131.85,390,0.0,2023-09-29,GOOG


In [26]:
# Microsoft
data = get_json('MSFT')
df_msft = format_json(data, 'MSFT')
df_msft.head(2)

<Response [200]>


,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,316.28,332.82,311.215,332.42,332.42,165,0.0,2023-10-11,MSFT
1,331.31,340.86,309.450,315.75,315.75,417,0.0,2023-09-29,MSFT


In [10]:
# IBM
data = get_json('IBM')
df_ibm = format_json(data, 'IBM')
df_ibm.head(2)

<Response [200]>


,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,140.04,143.4150,139.86,143.23,143.23,24,0.0,2023-10-11,IBM
1,147.26,151.9299,139.61,140.30,140.30,83,0.0,2023-09-29,IBM


In [11]:
# Se visualiza el tipo de dato de cada columna para la creación de la tabla
df_aapl.dtypes

open                      float64
high                      float64
low                       float64
close                     float64
adjusted close            float64
volume                      int32
dividend amount           float64
date               datetime64[ns]
symbol                     object
dtype: object

In [12]:
def cargar_en_redshift(conn, table_name, dataframe):
    # Funcion para cargar un dataframe en una tabla de redshift, creando la tabla si no existe
    # Definir formato tipos de datos SQL
    dtypes = dataframe.dtypes
    cols = list(dtypes.index)
    print(cols)
    tipos = list(dtypes.values)
    type_map = {
        'float64': 'FLOAT',
        'int32': 'INT',
        'datetime64[ns]': 'TIMESTAMP',
        'object': 'VARCHAR(255)'
    }
    # Definir formato TIPO_DATO revisando el tipo de dato de cada columna del dataframe
    sql_dtypes = [type_map.get(str(dtype), 'VARCHAR(255)') for dtype in tipos]

    # Definir formato COLUMNA TIPO_DATO
    column_defs = [f'"{name}" {data_type}' for name, data_type in zip(cols, sql_dtypes)]

    # Combina las columnas y los tipos de datos en una sola cadena de SQL para crear la tabla con todas la columnas necesarias
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(column_defs)});
        """

    # Crear la tabla
    cur = conn.cursor()
    try:
        # Se ejecuta el comando para crear la tabla creado anteriormente
        cur.execute(table_schema)

        # Generar los valores a insertar
        values = [tuple(x) for x in dataframe.values]

        # Definir el INSERT 
        insert_sql = f"INSERT INTO {table_name} (\"open\", \"high\", \"low\", \"close\", \"adjusted close\", \"volume\", \"dividend amount\", \"date\", \"symbol\") VALUES %s"

        # Execute the transaction to insert the data
        cur.execute("BEGIN")
        execute_values(cur, insert_sql, values)
        cur.execute("COMMIT")
        print('Proceso terminado')
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Rollback the transaction on error

def drop_table(conn, table_name):
    # Funcion para eliminar una tabla y asi poder volver a crearla sin problemas de sobreescritura
    cur = conn.cursor()
    try:
        cur.execute(f"DROP TABLE IF EXISTS {table_name}")
        conn.commit()
        print('Proceso terminado')
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Rollback the transaction on error


In [13]:
drop_table(conn=conn, table_name='apple_example')

Proceso terminado


In [14]:
# Prueba de la función de carga de datos con la tabla de solo el simbolo AAPL
cargar_en_redshift(conn=conn, table_name='apple_example', dataframe=df_aapl)

['open', 'high', 'low', 'close', 'adjusted close', 'volume', 'dividend amount', 'date', 'symbol']
Proceso terminado


# TERCERA PARTE (Unión de Datos y Subida a Redshift)

In [15]:
result_df = pd.concat([df_aapl, df_amzn, df_goog, df_msft, df_ibm], ignore_index=True)
result_df = result_df.sort_values(by=['date'], ascending=False)
result_df = result_df.reset_index(drop=True)
result_df.head(10)

,open,high,low,close,adjusted close,volume,dividend amount,date,symbol
0,171.220,179.8500,170.820,179.80,179.80,394,0.0,2023-10-11,AAPL
1,140.040,143.4150,139.860,143.23,143.23,24,0.0,2023-10-11,IBM
2,132.155,142.2200,132.065,141.70,141.70,155,0.0,2023-10-11,GOOG
3,316.280,332.8200,311.215,332.42,332.42,165,0.0,2023-10-11,MSFT
4,127.280,132.0500,124.130,131.83,131.83,352,0.0,2023-10-11,AMZN
5,331.310,340.8600,309.450,315.75,315.75,417,0.0,2023-09-29,MSFT
6,189.485,189.9800,167.620,171.21,171.21,1338,0.0,2023-09-29,AAPL
7,139.455,145.8600,123.040,127.12,127.12,1121,0.0,2023-09-29,AMZN
8,138.430,139.9300,128.190,131.85,131.85,390,0.0,2023-09-29,GOOG
9,147.260,151.9299,139.610,140.30,140.30,83,0.0,2023-09-29,IBM


In [16]:
drop_table(conn=conn, table_name='monthly_stocks_over_time')

Proceso terminado


In [17]:
cargar_en_redshift(conn=conn, table_name='monthly_stocks_over_time', dataframe=result_df)

['open', 'high', 'low', 'close', 'adjusted close', 'volume', 'dividend amount', 'date', 'symbol']
Proceso terminado
